In [2]:
import sys
import nltk
import sklearn
import pandas
import numpy

In [3]:
print("Python: {}".format (sys.version))
print("NLTK: {}".format(nltk.__version__))
print("SciKit-Learn: {}".format(sklearn.__version__))
print("Pandas: {}".format(pandas.__version__))
print("Numpy: {}".format(numpy.__version__))

Python: 3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]
NLTK: 3.2.5
SciKit-Learn: 1.0.2
Pandas: 1.3.5
Numpy: 1.21.5


# 1. Load the Dataset

In [4]:
import pandas as pd
import numpy as np

# load the dataset of the sms messages
df = pd.read_table('/content/drive/MyDrive/Text Classification/smsspamcollection/SMSSpamCollection', header=None, encoding='utf-8')

In [5]:
#print useful information about the dataset
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None
      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [6]:
#check class distribution
classes= df[0]
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64


# 2. Preprocess the Data

In [7]:
#convert class labels to binary values, 0 = ham, 1 = spam

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Y = encoder.fit_transform(classes)

print(classes[:10])
print(Y[:10])

0     ham
1     ham
2    spam
3     ham
4     ham
5    spam
6     ham
7     ham
8    spam
9    spam
Name: 0, dtype: object
[0 0 1 0 0 1 0 0 1 1]


In [8]:
#store the SMS message data

text_messages = df[1]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


###Metacharacter	Description
####^	Matches the starting position within the string. In  line-based tools, it matches the starting position of any line.

####.	Matches any single character (many applications exclude newlines, and exactly which characters are considered newlines is flavor-, character-encoding-, and platform-specific, but it is safe to assume that the line feed character is included). Within POSIX bracket expressions, the dot character matches a literal dot. For example, a.c matches "abc", etc., but [a.c] matches only "a", ".", or "c".

####[ ]	A bracket expression. Matches a single character that is contained within the brackets. For example, [abc] matches "a", "b", or "c". [a-z] specifies a range which matches any lowercase letter from "a" to "z". These forms can be mixed: [abcx-z] matches "a", "b", "c", "x", "y", or "z", as does [a-cx-z].
#The - character is treated as a literal character if it is the last or the first (after the ^, if present) character within the brackets: [abc-], [-abc]. Note that backslash escapes are not allowed. The ] character can be included in a bracket expression if it is the first (after the ^) character: []abc].

####[^ ]	Matches a single character that is not contained within the brackets. For example, [^abc] matches any character other than "a", "b", or "c". [^a-z] matches any single character that is not a lowercase letter from "a" to "z". Likewise, literal characters and ranges can be mixed.

####$	Matches the ending position of the string or the position just before a string-ending newline. In line-based tools, it matches the ending position of any line.

####( )	Defines a marked subexpression. The string matched within the parentheses can be recalled later (see the next entry,\n). A marked subexpression is also called a block or capturing group. BRE mode requires \( \).

####\n	Matches what the nth marked subexpression matched, where n is a digit from 1 to 9. This construct is vaguely defined in the POSIX.2 standard. Some tools allow referencing more than nine capturing groups. Also known as a backreference. backreferences are only supported in BRE mode

####*	Matches the preceding element zero or more times. For example, ab*c matches "ac", "abc", "abbbc", etc. [xyz]* matches "", "x", "y", "z", "zx", "zyx", "xyzzy", and so on. (ab)* matches "", "ab", "abab", "ababab", and so on.

####{m,n}	Matches the preceding element at least m and not more than n times. For example, a{3,5} matches only "aaa", "aaaa", and "aaaaa". This is not found in a few older instances of regexes. BRE mode requires \{m,n\}.


In [9]:
#use regular expressions to replace email addresses, urls, phone numbers,other numbers, symbols

#replace email address with 'emailaddr'

processed = text_messages.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddr')

#replace urls with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]*\.{a-zA-Z}{2,3}(/\S*)?$', 'webaddress')

#replace money symbols with 'moneysymb'£|\$
processed = processed.str.replace(r'£|\$', 'moneysymb')

#replace 10 digit phone numbers with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumber')

#replace normal numbers with 'number'
processed = processed.str.replace(r'\d+(\.\d+)?', 'number')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: The default value of regex will change from True to False in a future version.


In [10]:
#remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

#replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

#replace leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [11]:
#change words to lower case - Hello,  HELLO, hello are all the same word:
processed = processed.str.lower()
print(processed)

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in number a wkly comp to win fa cup...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the numbernd time we have tried number...
5568                  will ü b going to esplanade fr home
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: 1, Length: 5572, dtype: object


In [12]:
#remove stop words from text messages

from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
#remove word stems using a Porter stemmer
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))

In [14]:
print(processed)

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri number wkli comp win fa cup final t...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
                              ...                        
5567    numbernd time tri number contact u u moneysymb...
5568                              ü b go esplanad fr home
5569                                    piti mood suggest
5570    guy bitch act like interest buy someth els nex...
5571                                       rofl true name
Name: 1, Length: 5572, dtype: object


In [15]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
#create a bag-of-words
all_words = []

for message in processed:
  words = word_tokenize(message)
  for w in words:
    all_words.append(w)

all_words = nltk.FreqDist(all_words)    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
#print the total number of words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(20)))

Number of words: 6574
Most common words: [('number', 2759), ('u', 1207), ('call', 674), ('go', 456), ('get', 451), ('ur', 391), ('gt', 318), ('lt', 316), ('come', 304), ('moneysymbnumb', 303), ('ok', 293), ('free', 284), ('day', 276), ('know', 275), ('love', 266), ('like', 261), ('got', 252), ('time', 252), ('good', 248), ('want', 247)]


In [17]:
#use the 2000 most common words as features
 
word_features = list(all_words.keys())[:2000]

In [18]:
#define a find_features function
def find_features(message):
  words = word_tokenize(message)
  features = {}
  for word in word_features:
    features[word] = (word in words)
    
  return features 

#Let's see an example
features = find_features(processed[0])
for key, value in features.items():
  if value == True:  
    print(key)

go
jurong
point
crazi
avail
bugi
n
great
world
la
e
buffet
cine
got
amor
wat


In [19]:
processed[0]

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [20]:
features

{'go': True,
 'jurong': True,
 'point': True,
 'crazi': True,
 'avail': True,
 'bugi': True,
 'n': True,
 'great': True,
 'world': True,
 'la': True,
 'e': True,
 'buffet': True,
 'cine': True,
 'got': True,
 'amor': True,
 'wat': True,
 'ok': False,
 'lar': False,
 'joke': False,
 'wif': False,
 'u': False,
 'oni': False,
 'free': False,
 'entri': False,
 'number': False,
 'wkli': False,
 'comp': False,
 'win': False,
 'fa': False,
 'cup': False,
 'final': False,
 'tkt': False,
 'numberst': False,
 'may': False,
 'text': False,
 'receiv': False,
 'question': False,
 'std': False,
 'txt': False,
 'rate': False,
 'c': False,
 'appli': False,
 'numberovernumb': False,
 'dun': False,
 'say': False,
 'earli': False,
 'hor': False,
 'alreadi': False,
 'nah': False,
 'think': False,
 'goe': False,
 'usf': False,
 'live': False,
 'around': False,
 'though': False,
 'freemsg': False,
 'hey': False,
 'darl': False,
 'week': False,
 'word': False,
 'back': False,
 'like': False,
 'fun': False,
 

In [21]:
#find features for all messages

messages = list(zip(processed, Y))

#defone a seed for reproducibility
seed = 1
np.random.seed = seed
np.random.shuffle(messages)

#call find_features function for each SMS messages
featuresets = [(find_features(text), label) for (text, label) in messages]

In [22]:
#split training and testing data sets using sklearn
from sklearn import model_selection
training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state= seed)

In [23]:
print('Training: {}'.format(len(training)))
print('Testing: {}'.format(len(testing)))

Training: 4179
Testing: 1393


#4. Scikit-Learn Classifiers with NLTK

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [26]:
#Define the models to train

names = ['K Nearest Neighbors', 'Decision Tree', 'Random Forest', 'Logistic Regression', 'SGD Classifier', 'Naive Bayes', 'SVM Linear']

classifiers = [
              KNeighborsClassifier(),
              DecisionTreeClassifier(),
              RandomForestClassifier(),
              LogisticRegression(),
              SGDClassifier(max_iter = 100),
              MultinomialNB(),
              SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
print(models)

[('K Nearest Neighbors', KNeighborsClassifier()), ('Decision Tree', DecisionTreeClassifier()), ('Random Forest', RandomForestClassifier()), ('Logistic Regression', LogisticRegression()), ('SGD Classifier', SGDClassifier(max_iter=100)), ('Naive Bayes', MultinomialNB()), ('SVM Linear', SVC(kernel='linear'))]


In [27]:
#Wrap models in NLTK
from nltk.classify.scikitlearn import SklearnClassifier


for name, model in models:
  nltk_model = SklearnClassifier(model)
  nltk_model.train(training)
  accuracy = nltk.classify.accuracy(nltk_model, testing) * 100
  print('{}: Accuracy: {}'.format(name, accuracy))

K Nearest Neighbors: Accuracy: 94.11342426417804
Decision Tree: Accuracy: 97.27207465900933
Random Forest: Accuracy: 98.92318736539842
Logistic Regression: Accuracy: 98.85139985642498
SGD Classifier: Accuracy: 98.42067480258436
Naive Bayes: Accuracy: 98.42067480258436
SVM Linear: Accuracy: 98.77961234745155


#Eventhough I have got very good results, I am still doing ensemble learning to learn.


In [41]:
#ensemble method - Voting Classifier
from sklearn.ensemble import VotingClassifier

#Define the models to train

names = ['K Nearest Neighbors', 'Decision Tree', 'Random Forest', 'Logistic Regression', 'SGD Classifier', 'Naive Bayes', 'SVM Linear']

classifiers = [
              KNeighborsClassifier(),
              DecisionTreeClassifier(),
              RandomForestClassifier(),
              LogisticRegression(),
              SGDClassifier(max_iter = 100),
              MultinomialNB(),
              SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier (estimators = models, voting= 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_ensemble, testing) * 100
print('Ensemble Method Accuracy: {}'.format(accuracy))


Ensemble Method Accuracy: 98.77961234745155


In [43]:
#make class label prediction for testing set
txt_features, labels = list(zip(*testing))

prediction = nltk_ensemble.classify_many(txt_features)

In [46]:
#print a confusion matrix and a classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['ham', 'spam']],
    columns = [['predicted', 'predicted'], ['ham', 'spam']]
)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1216
           1       0.98      0.93      0.95       177

    accuracy                           0.99      1393
   macro avg       0.98      0.96      0.97      1393
weighted avg       0.99      0.99      0.99      1393



predicted     
                  ham spam
actual ham       1212    4
       spam        13  164